# Project B


This notebook will cover only cover the data prep and merge of the two larger json files.

## Packages Needed

In [1]:
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
import os

In [2]:
path = 'C:\\Users\\Miche\\OneDrive - Danmarks Tekniske Universitet\\MMC\\2. Semester\\Social Data\\websites\\data'
os.chdir(path)
os.getcwd()

'C:\\Users\\Miche\\OneDrive - Danmarks Tekniske Universitet\\MMC\\2. Semester\\Social Data\\websites\\data'

## Load Data Business Dataset + Prep

In [3]:
df = pd.read_json('yelp_academic_dataset_business.json', lines=True)
print(df.shape)

(160585, 14)


In [4]:
df.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

We only want to look at the restaurants that are open and not closed.

In [5]:
df = df[df['is_open']==1]

In [6]:
df= df.drop(['is_open'], axis=1)

Start by select all the rows that mention restaurants.

In [7]:
df = df[df['categories'].str.contains('Restaurants',
              case=False, na=False)]

In [8]:
df_explode = df.assign(categories = df.categories
                         .str.split(', ')).explode('categories')

In [9]:
df_explode.categories.value_counts().head(20)

Restaurants                  32022
Food                         10732
Nightlife                     5550
Bars                          5345
Sandwiches                    4795
American (Traditional)        4379
Fast Food                     4326
Pizza                         3890
Breakfast & Brunch            3872
American (New)                3254
Burgers                       3246
Coffee & Tea                  3010
Mexican                       2815
Chinese                       2329
Italian                       2130
Seafood                       1998
Salad                         1975
Event Planning & Services     1916
Cafes                         1871
Japanese                      1773
Name: categories, dtype: int64

In [10]:
remove = ['Food Delivery Services','Food Safety Training', 'Food Tours', 
          'Food Banks','Chinese Martial Arts','Traditional Chinese Medicine',
         'Coffee & Tea Supplies','Japanese Curry', 'Food Court',
         'Food Trucks','Food Stands']

df = df[~df['categories'].str.contains('|'.join(remove),
              case=False, na=False)]

Try to split the category column into different subgroups, so we can do some nice plotting.

In [11]:
cat_kitchens = ['thai', 'chinese','japanese','korean','indian','american',
                 'caribbean','italian','mediterranean','mexican', 'cajun',
                'vietnamese','greek']

cat_type = ['Food','Nightlife','Bars','Sandwiches','Pizza','Breakfast & Brunch', 'Fast Food',
            'Burgers','Salad', 'Buffet', 'Cafes','Coffee & Tea', 'Vegetarian', 'Steakhouse', 'Sushi Bars',
            'Diners','Wine Bars']

In [12]:
df_explode = df.assign(categories = df.categories
                         .str.split(', ')).explode('categories')

In [13]:
df_cat_kitchen = df_explode[df_explode['categories'].str.contains(
              '|'.join(cat_kitchens),
              case=False, na=False)]

df_cat_kitchen = df_cat_kitchen.rename(columns={'categories':'cat_kitchen'})
df_cat_kitchen = df_cat_kitchen['cat_kitchen']

In [14]:
df_cat_type = df_explode[df_explode['categories'].str.match(
              '|'.join(cat_type),
              case=True, na=False)]

df_cat_type = df_cat_type.rename(columns={'categories':'cat_type'})
df_cat_type = df_cat_type['cat_type']

In [15]:
df_cat_type.unique()

array(['Food', 'Bars', 'Nightlife', 'Salad', 'Sandwiches', 'Cafes',
       'Vegetarian', 'Breakfast & Brunch', 'Pizza', 'Sushi Bars',
       'Steakhouses', 'Fast Food', 'Burgers', 'Coffee & Tea', 'Diners',
       'Buffets', 'Wine Bars'], dtype=object)

In [16]:
df_merge_list = [df, df_cat_kitchen, df_cat_type]

In [17]:
df_merged = reduce(lambda  left, right: pd.merge(left,right,left_index=True,
                                            right_index=True), df_merge_list)

In [18]:
df_merged.shape

(29836, 15)

In [19]:
df_merged.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours,cat_kitchen,cat_type
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",American (Traditional),Food
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",American (Traditional),Bars
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",American (Traditional),Nightlife
13,ufCxltuh56FF4-ZFZ6cVhg,Sister Honey's,247 E Michigan St,Orlando,FL,32806,28.513265,-81.374707,4.5,135,"{'BusinessParking': '{'garage': False, 'street...","Restaurants, American (New), Bakeries, Dessert...","{'Tuesday': '11:0-18:0', 'Wednesday': '11:0-18...",American (New),Food
19,dmbbf3AqeG61_QHRZi1M1w,RaceTrac,350 W Sand Lake Rd,Pine Castle,FL,32809,28.450302,-81.380587,3.5,5,"{'BikeParking': 'False', 'RestaurantsPriceRang...","Automotive, American (Traditional), Gas Statio...",None,American (Traditional),Food


## Load Review Data (Massive dataset)

In [20]:
size = 500000
review = pd.read_json('yelp_academic_dataset_review.json', lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [21]:
# There are multiple chunks to be read
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(df_merged, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

416263 out of 500,000 related reviews
384085 out of 500,000 related reviews
371677 out of 500,000 related reviews
358510 out of 500,000 related reviews
349565 out of 500,000 related reviews
356785 out of 500,000 related reviews
353610 out of 500,000 related reviews
407361 out of 500,000 related reviews
371185 out of 500,000 related reviews
362734 out of 500,000 related reviews
361974 out of 500,000 related reviews
362709 out of 500,000 related reviews
388455 out of 500,000 related reviews
373812 out of 500,000 related reviews
414622 out of 500,000 related reviews
414618 out of 500,000 related reviews
433111 out of 500,000 related reviews
107384 out of 500,000 related reviews


In [22]:
df.shape

(6588460, 19)

## More data prep

Select 2014-2021, so we have full years

In [23]:
print (df.date.min())
print (df.date.max())

2004-10-14 02:57:52
2021-01-28 15:23:52


In [24]:
df.date.dtypes

dtype('O')

In [25]:
df['date'] = pd.to_datetime(df['date'])

In [26]:
df.date.dtypes

dtype('<M8[ns]')

In [27]:
time_start = pd.Timestamp(2014,1,1)

In [28]:
#Filter the data
df = df.loc[(df['date']>= time_start)]

In [29]:
print (df.date.min())
print (df.date.max())

2014-01-01 00:01:51
2021-01-28 15:23:52


In [30]:
df.shape

(5177322, 19)

Save to csv file

In [31]:
path = 'C:\\Users\\Miche\\OneDrive - Danmarks Tekniske Universitet\\MMC\\2. Semester\\Social Data\\websites\\Restaurant-Guide'
os.chdir(path)
os.getcwd()

'C:\\Users\\Miche\\OneDrive - Danmarks Tekniske Universitet\\MMC\\2. Semester\\Social Data\\websites\\Restaurant-Guide\\data'

In [32]:
csv_name = "yelp_reviews_RV_categories.csv"
df.to_csv(csv_name, index=False)

In [33]:
df.shape

(5177322, 19)